In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from shapely.geometry import MultiPolygon, Polygon, mapping
import geopandas as gpd

In [2]:
parcels = gpd.read_file('/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/parcels.shp')
geometries = parcels.geometry.values

In [4]:
from openeo.local import LocalConnection
local_conn = LocalConnection("./")

In [6]:
bbox = parcels.total_bounds
bbox

array([10.93311286, 46.66029087, 11.41500808, 46.85360212])

In [8]:
# import json

# # Specify the path to the GeoJSON file
# input_geojson_file = 'parcels.geojson'

# # Read GeoJSON file as a regular JSON file
# with open(input_geojson_file, 'r') as file:
#     json_data = json.load(file)

In [9]:
# features = json_data['features']

In [10]:
# for feature in json_data['features']:
#     print(feature['geometry']['coordinates'])
#     break

#from shapely.geometry import MultiPolygon, Polygon, mapping
#??mapping

In [12]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a"
spatial_extent =  {"west": bbox[0], "south": bbox[1], "east": bbox[2], "north": bbox[3], 'crs':'EPSG:4326'}

temporal_extent = ["2022-06-01", "2022-06-30"]
bands = ["B04","B03","B02","B08"] # B04 = red, B03 = green, B02 = blue, B08 = near infrared
properties = {"eo:cloud_cover": dict(lt=30)}


datacube = local_conn.load_stac(
    url=url,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent,
    bands=bands,
    properties=properties,
)

# Geometries - Polygon
datacube = datacube.filter_spatial(geometries[0])

datacube

/home/malasawedah/.conda/envs/openeo/lib/python3.10/site-packages/stackstac/prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [13]:
datacube_xr = datacube.execute()
datacube_xr

/home/malasawedah/.conda/envs/openeo/lib/python3.10/site-packages/stackstac/prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.Dataset>
Dimensions:                                     (time: 7, band: 4, y: 2250,
                                                 x: 3739)
Coordinates: (12/44)
  * time                                        (time) datetime64[ns] 2022-06...
    id                                          (time) <U54 'S2A_MSIL2A_20220...
  * band                                        (band) <U3 'B04' 'B03' ... 'B08'
  * x                                           (x) float64 6.474e+05 ... 6.8...
  * y                                           (y) float64 5.192e+06 ... 5.1...
    constellation                               <U10 'Sentinel 2'
    ...                                          ...
    title                                       (band) <U20 'Band 4 - Red - 1...
    gsd                                         float64 10.0
    common_name                                 (band) <U5 'red' ... 'nir'
    center_wavelength                           (band) float64 0.665 ... 0.842
    full_width_half_max                         (band) float64 0.038 ... 0.145
    epsg                                        int64 32632
Data variables:
    stackstac-4a2654c3011dc2d5a001281950e49702  (time, band, y, x) float64 dask.array<chunksize=(1, 1, 432, 383), meta=np.ndarray>

### Feature collection - Mulitble polygons

In [14]:
# import json
# input_geojson_file = 'parcels.geojson'

# with open(input_geojson_file, 'r') as file:
#     json_data = json.load(file)

In [15]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a"
spatial_extent =  {"west": bbox[0], "south": bbox[1], "east": bbox[2], "north": bbox[3], 'crs':'EPSG:4326'}

temporal_extent = ["2022-06-01", "2022-06-30"]
bands = ["B04","B03","B02","B08"] # B04 = red, B03 = green, B02 = blue, B08 = near infrared
properties = {"eo:cloud_cover": dict(lt=30)}


datacube = local_conn.load_stac(
    url=url,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent,
    bands=bands,
    properties=properties,
)

# Feature collection
datacube = datacube.filter_spatial(MultiPolygon(geometries))

datacube

/home/malasawedah/.conda/envs/openeo/lib/python3.10/site-packages/stackstac/prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [16]:
datacube_xr = datacube.execute()
datacube_xr

/home/malasawedah/.conda/envs/openeo/lib/python3.10/site-packages/stackstac/prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.Dataset>
Dimensions:                                     (time: 7, band: 4, y: 2250,
                                                 x: 3739)
Coordinates: (12/44)
  * time                                        (time) datetime64[ns] 2022-06...
    id                                          (time) <U54 'S2A_MSIL2A_20220...
  * band                                        (band) <U3 'B04' 'B03' ... 'B08'
  * x                                           (x) float64 6.474e+05 ... 6.8...
  * y                                           (y) float64 5.192e+06 ... 5.1...
    constellation                               <U10 'Sentinel 2'
    ...                                          ...
    title                                       (band) <U20 'Band 4 - Red - 1...
    gsd                                         float64 10.0
    common_name                                 (band) <U5 'red' ... 'nir'
    center_wavelength                           (band) float64 0.665 ... 0.842
    full_width_half_max                         (band) float64 0.038 ... 0.145
    epsg                                        int64 32632
Data variables:
    stackstac-0d73ff8146c7238a60a53047227ce0b7  (time, band, y, x) float64 dask.array<chunksize=(1, 1, 432, 383), meta=np.ndarray>

In [18]:
# multi_polygon = MultiPolygon([polygon1, polygon2])

# # Use mapping to convert the MultiPolygon to a GeoJSON-like mapping
# geojson_mapping = mapping(multi_polygon)

In [19]:
# import shapely
# geojson = {'type': 'Polygon', 'coordinates': [[[11.187849533252663, 46.76775897917851], [11.187867553984386, 46.76776218032945], [11.187843375588937, 46.7676962082461], [11.187843739846272, 46.76769614728225], [11.187842919506826, 46.767693679102095], [11.187842814808828, 46.76769368110453], [11.18784034104483, 46.76766397597691], [11.18782314440813, 46.76766430487451], [11.187815590439431, 46.767663414402826], [11.187812618072035, 46.76766213932076], [11.187812430505938, 46.76766235889676], [11.187777740451454, 46.7676479301481], [11.187775067407054, 46.7676527510206], [11.187761570239095, 46.76767715488998], [11.18774320153422, 46.76770868955651], [11.187849533252663, 46.76775897917851]]]}
# polygon = shapely.geometry.Polygon(geojson['coordinates'][0])